In [21]:
#import modules
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import time

#FUNCTIONS
#function that takes desired variables with your list of dictionaries scraped and returns a clean list of these variables
def get_vars(stats, vars_wanted):
    stats_vars = []
    items=[]
    #iterate through scraped data to find desired variables
    for list_item in stats:
        dict_item = [key for key,value in list_item.items()]
        if dict_item[0] in vars_wanted:
            #check for duplicates
            if dict_item not in items:
                items.append(dict_item)
                stats_vars.append(list_item)
    return stats_vars


#---------------------------------------------------------------------------------------------------------------------------
#MAIN

#URL of page to be scraped
url = 'https://www.pgatour.com/players.html'

#Retrieve page with the requests module
response = requests.get(url)

#create BeautifulSoup object; parse w 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')


#SCRAPE LINK TO EACH PLAYER STAT PAGE***********************************
#get names of all the player links
#retrieve the parent divs for all links
#players = soup.find_all('span',class_="name")

#create empty list to accept data
player_names = []
player_url = []

#loop through each parent div and grab the link to the player stat page
for player in players[player_count:]:
    #get name of player
    player_names.append(player.a.text)
    #get url for player performance page
    player_url.append(player.a['href'])


#SCRAPE PGA PERFORMANCE DATA FOR EACH INDIVIDUAL PLAYER******************
base_url = 'https://www.pgatour.com'

#execute chromedriver
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

#master data set
#DATA = []
#keep track of player count
#player_count = 0
#keep track of the players unable to scrape
#error_player = []

for player,url in zip(player_names,player_url):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=True)
    time.sleep(2)
    
    scrape_url = base_url + url
    print("scraped URL")

    #SPLINTER
    print("try visit url")
    browser.visit(scrape_url)
    time.sleep(8)
    print("done visit url")
    
    #if pop-up add comes up, wait it out
    try:
        print("try click performance")
        browser.click_link_by_partial_text("Performance")
        time.sleep(2)
        print("done click performance")
    except:
        print("except try click performance")
        time.sleep(10)
        browser.click_link_by_partial_text("Performance")
        time.sleep(2)
        print("done click performance except")

    try:
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')   
        
        #INTRO/PLAYER INFO DATA########################################################################
        #photo of player
        try:
            photo_url = soup.find('img', class_='photo')['src']
            player_intro = {'Player Name': player.replace('\xa0',' '), 'photo_url': photo_url}
        except TypeError:
            player_intro = {'Player Name': player.replace('\xa0',' '), 'photo_url': 'none'}

        #scrape all html hstat code
        info = soup.findAll('div', class_='item')

        #headline stats
        info_stats = []

        for i in info:
            try:
                #get stat name
                caption = i.find('div',class_='denotation').text
                #get stat value
                value = i.find('div', class_='value').text
                #clean up text
                value = value.replace('\xa0','')
                value = value.replace('\n','')
                #save to dictionary
                post = {'caption':caption, 'value': value}
                #append to list
                info_stats.append(post)
            except AttributeError:
                nothing = 0
                
        ####TOURNAMENT DATA###########################################################################
        bla = soup.findAll('div', class_ = 'holder')
        dates_ = soup.findAll('td', class_ = 'date')
        rounds = soup.findAll('td', class_='round')

        tourney_name = []
        all_text = []
        scores = []
        to_par = []
        pos = []
        dates = []

        for i in bla:
            x = i.find('tbody')
            #tourney info
            tourneys = x.findAll('p')
            #need this for all text
            tds = x.findAll('td')
            #get all text to use later for pos
            [all_text.append(td.text) for td in tds]
            
            #tournament names
            [tourney_name.append(j.text) for j in tourneys]

        #clean dates
        [dates.append(d.text) for d in dates_]
        #scores of each round in increments of 4 ('--' means no score)
        [scores.append(r.text) for r in rounds]
        #now append tournament position results by getting list item after tournament name
        [pos.append(all_text[all_text.index(tourney)+1]) for tourney in tourney_name]
        #now append tournament position results by getting list item after tournament name
        [to_par.append(all_text[all_text.index(tourney)+8]) for tourney in tourney_name]

        #create final dictionary of tournaments for the past year
        tournament_history = []
        for date,tourney,score,rank in zip(dates,tourney_name,to_par,pos):
            try:
                #create dictionary with all info
                post = {'Date':date, 
                        'Tournament Name':tourney, 
                        'Total Score':score, 
                        'POS':rank}
                #append to final list
                tournament_history.append(post)
            except AttributeError:
                nothing=0
        
        #PERFORMANCE DATA##############################################################################
        
        dropdown = browser.find_by_xpath("//div[@class='select year']//select[@class='hasCustomSelect']")
        h_stats_all = []
        a_stats_all = []
        extra_stats_var_all = []
        for option in dropdown.find_by_tag('option'):  
            option.click()
            time.sleep(2)
            try:
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
            except:
                time.sleep(10)
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')

            #scrape all html hstat code
            hstats = soup.findAll('div', class_='stat')

            #headline stats
            h_stats = []
            year = {"year":option.text}
            h_stats.append(year)

            for hstat in hstats:
                try:
                    #get stat name
                    caption = hstat.find('div',class_='caption').text
                    #get stat value
                    value = hstat.find('div',class_='value').text

                    #save to dictionary
                    #post = {'caption':caption, 'value': value}
                    post = {caption : value}

                    #append to list
                    h_stats.append(post)
                except AttributeError:
                    nothing = 0

            h_stats_all.append(h_stats)

            #scrape all html astat code
            astats = soup.findAll('tr')

            #attribute stats
            a_stats = []
            year = {"year":option.text}
            a_stats.append(year)

            for astat in astats:
                try:
                    #get the stat name
                    caption = astat.find('td',class_='caption').text
                    #get stat value
                    value = astat.find('td',class_='value').text

                    #save to dictionary
                    post = {caption : value}

                    #append to list
                    a_stats.append(post)
                except AttributeError:
                    nothing = 0

            a_stats_all.append(a_stats)

            #scrape for additional needed info
            extrastats = soup.findAll('td')

            #attribute stats
            extra_stats = []

            for extra in extrastats:
                try:
                    #get the stat name
                    text = extra.text
                    #append to list
                    extra_stats.append(text)
                except AttributeError:
                    nothing=0

            #these attributes are unique with no captions/values-All string format
            #search sub_strings of desired variables for values
            sub_strings = ['Total Left rough', 'Total Right rough', 'Possible Fwys', 'Distance Rank', 'Accuracy Rank',
                  'Total Club Head Speed', 'Total Attempts']
            extra_stats_var= []
            year = {"year":option.text}
            extra_stats_var.append(year)
            for sub in sub_strings:
                x = [s for s in extra_stats if sub in s]
                if x:
                    x = x[0].split(':')
                    x[1] = x[1].replace(' ','')

                    #post = {'caption': x[0], 'value': x[1]}
                    post = {x[0] : x[1]}
                    extra_stats_var.append(post)
                else:
                    nothing=0

            extra_stats_var_all.append(extra_stats_var)

        #variables wanted from info stats
        info_var = ['year','Height', 'Weight', 'AGE', 'Turned Pro', 'College', 'Birthplace' ,'FEDEXCUP Rank', 'FEDEXCUP Points', 'Scoring Average']
        #variables wanted from headline stats
        h_var = ['year','Total Distance', 'Total Drives', '# of Drives', 'Fairways Hit', 'Possible Fairways', 'Measured Rounds']
        #variables wanted from additional stats
        a_var = ['year','Driving Distance','Driving Accuracy Percentage','Total Driving','Club Head Speed',
                'Distance from Edge of Fairway','Left Rough Tendency','Right Rough Tendency','Total Driving Efficiency']
        extra_var = ['year','Total Left rough', 'Total Right rough', 'Possible Fwys', 'Distance Rank', 'Accuracy Rank',
            'Total Club Head Speed', 'Total Attempts']
        
        #get only desired variables listed above
        info_stats_vars = get_vars(info_stats, info_var)
        h_stats_vars = [get_vars(h_stats, h_var) for h_stats in h_stats_all]
        a_stats_vars = [get_vars(a_stats, a_var) for a_stats in a_stats_all]
        extra_stats_vars = [get_vars(extra_stats, extra_var) for extra_stats in extra_stats_var_all]
        
        #convert lists with dictionaries to 1 list with big dictionary
        info_stats_vars = {key: value for info_stat in info_stats_vars for key, value in info_stat.items()}
        #all years of stats
        a_stats_vars = [{key: value for a_stat in list_item for key, value in a_stat.items()} for list_item in a_stats_vars]
        h_stats_vars = [{key: value for a_stat in list_item for key, value in a_stat.items()} for list_item in h_stats_vars]
        extra_stats_vars = [{key: value for a_stat in list_item for key, value in a_stat.items()} for list_item in extra_stats_vars]
        
        #combine all statistic variables into one list of dictionaries
        all_stat_var =[]
        for a_stat, h_stat, extra_stat in zip(a_stats_vars, h_stats_vars, extra_stats_vars):
            temp_dict = {}
            temp_dict.update(a_stat)
            temp_dict.update(h_stat)
            temp_dict.update(extra_stat)

            all_stat_var.append(temp_dict)
    
        #output list
        player_final = {}
        player_final['player_intro'] = player_intro
        player_final['info'] = info_stats_vars
        player_final['all_stat_var'] = all_stat_var
        player_final['tournament_hist'] = tournament_history

        DATA.append(player_final)
        print(f"{player_count} Player Data Successfully Acquired!: {player}")
        player_count += 1
        browser.quit()
    
    except:
        error_player.append(player_count)
        print(f"{player_count} !!!!!!!!!!!Data Retrieval Unsuccessful: {player}")
        player_count += 1
        browser.quit()
        
    
    
#PART 2: TITLEIST WEB SCRAPE
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://www.titleist.com/tour/pga/players"
base_url = "https://www.titleist.com"
browser.visit(url)
time.sleep(2)

#close pop-up
try:
    browser.find_by_css("Button")[11].click()
except:
    print("No Pop Up")
    
#info we will scrape
done = 0
titleist_players = []

while done != 1:
    try:
        
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        
        #scrape all html hstat code
        test = soup.findAll('li', class_='m-results-item')
        
        for t in test:

            #name of player
            n = t.find('h2', class_='m-results-label').text
            #format name to match other data scraped
            n = n.replace(' ', '')
            n = n.replace('\n', '')
            isupper = [letter.isupper() for letter in n]
            n = n + ", "
            #index of last name starting
            last_name_index = [i for i, x in enumerate(isupper) if x][1]
            #break up
            first_name = n[0:last_name_index]
            last_name = n[last_name_index:]
            n = last_name + first_name
            name = n

            #equipment using
            e = t.find('em').text
            #name.append(n)
            equipment = e

            if e == 'Brand Ambassador':
                player_page = t.find('h2', class_='m-results-label').a['href']
                club_url = base_url + player_page
                browser.visit(club_url)
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                try:
                    driver = soup.find('div', class_ = "m-category-listing-content").a.text
                    driver = driver.replace(" ", "")
                    driver = driver.replace("\n", "")
                    type_equipment = driver
                    browser.back()
                except:
                    golf_ball = t.findAll('p')
                    ball = [b.text for b in golf_ball]
                    ball = ball[1]
                    ball = ball.replace("Brand Ambassador", "")
                    type_equipment = ball
                    browser.back()
            else:
                golf_ball = t.findAll('p')
                ball = [b.text for b in golf_ball]
                ball = ball[1]
                ball = ball.replace("Golf Ball Player", "")
                type_equipment = ball
            
            #put needed info to dictionary
            post = {'Player Name': n, 
            'Type': equipment, 
            'Equipment': type_equipment, 
            }
            
            #append each player to list
            titleist_players.append(post)
                    
        browser.click_link_by_partial_text("Next")
        time.sleep(1)
    except:
        done=1
        
browser.quit()

scraped URL
try visit url
done visit url
try click performance
done click performance
600 Player Data Successfully Acquired!: Poulter, Ian
scraped URL
try visit url
done visit url
try click performance
done click performance
601 Player Data Successfully Acquired!: Power, Seamus
scraped URL
try visit url
done visit url
try click performance
done click performance
602 Player Data Successfully Acquired!: Powers, Greg
scraped URL
try visit url
done visit url
try click performance
done click performance
603 Player Data Successfully Acquired!: Price, Nick
scraped URL
try visit url
done visit url
try click performance
done click performance
604 Player Data Successfully Acquired!: Pride, Dicky
scraped URL
try visit url
done visit url
try click performance
except try click performance
done click performance except
605 Player Data Successfully Acquired!: Prugh, Alex
scraped URL
try visit url
done visit url
try click performance
done click performance
606 Player Data Successfully Acquired!: Purdy

done visit url
try click performance
except try click performance
done click performance except
657 Player Data Successfully Acquired!: Shaw, Tom
scraped URL
try visit url
done visit url
try click performance
done click performance
658 Player Data Successfully Acquired!: Shearer, Bob
scraped URL
try visit url
done visit url
try click performance
done click performance
659 Player Data Successfully Acquired!: Shih-chang, Chan
scraped URL
try visit url
done visit url
try click performance
done click performance
660 Player Data Successfully Acquired!: Short, Jr., Wes
scraped URL
try visit url
done visit url
try click performance
done click performance
661 Player Data Successfully Acquired!: Sieckmann, Tom
scraped URL
try visit url
done visit url
try click performance
done click performance
662 Player Data Successfully Acquired!: Siem, Marcel
scraped URL
try visit url
done visit url
try click performance
done click performance
663 Player Data Successfully Acquired!: Sifford, Charles
scraped

713 Player Data Successfully Acquired!: Streelman, Kevin
scraped URL
try visit url
done visit url
try click performance
done click performance
714 Player Data Successfully Acquired!: Stricker, Steve
scraped URL
try visit url
done visit url
try click performance
done click performance
715 Player Data Successfully Acquired!: Stroud, Chris
scraped URL
try visit url
done visit url
try click performance
done click performance
716 Player Data Successfully Acquired!: Stuard, Brian
scraped URL
try visit url
done visit url
try click performance
except try click performance
done click performance except
717 Player Data Successfully Acquired!: Sturgeon, Grant
scraped URL
try visit url
done visit url
try click performance
done click performance
718 Player Data Successfully Acquired!: Sucher, Zack
scraped URL
try visit url
done visit url
try click performance
except try click performance
done click performance except
719 Player Data Successfully Acquired!: Sullivan, Andy
scraped URL
try visit url
d

scraped URL
try visit url
done visit url
try click performance
done click performance
771 Player Data Successfully Acquired!: Waite, Grant
scraped URL
try visit url
done visit url
try click performance
done click performance
772 Player Data Successfully Acquired!: Waldorf, Duffy
scraped URL
try visit url
done visit url
try click performance
except try click performance
done click performance except
773 Player Data Successfully Acquired!: Walker, Jimmy
scraped URL
try visit url
done visit url
try click performance
done click performance
774 Player Data Successfully Acquired!: Wallace, Matt
scraped URL
try visit url
done visit url
try click performance
done click performance
775 Player Data Successfully Acquired!: Warren, Marc
scraped URL
try visit url
done visit url
try click performance
done click performance
776 Player Data Successfully Acquired!: Watney, Nick
scraped URL
try visit url
done visit url
try click performance
done click performance
777 Player Data Successfully Acquired!: 

In [ ]:
#get all player names of each list in DATA
DATA_player_names = [d['player_intro']['Player Name'] for d in DATA]

In [ ]:
#add titleist player info to DATA set
for player in titleist_players:
    #get name to match to master data set
    name = player['Player Name']
    print(name)
    #find which item in master data set this name matches
    index = [i for i, s in enumerate(DATA_player_names) if name in s]
    print(index)
    #create new list item/dictionary for only titleist players in master data set
    #DATA[index[0]]['Titleist'] = player

In [ ]:
#APPEND TEST

In [ ]:
DATA_copy = DATA[:]
titleist_copy = titleist_players[:]

In [ ]:
DATA_copy = DATA_copy[737]
DATA_copy

In [ ]:
player_test = titleist_copy[0]
player_test

In [ ]:
DATA_copy['player_intro']['Player Name']

In [ ]:
player_test['Player Name']

In [ ]:
DATA_copy['player_intro']['Player Name'] == player_test['Player Name']

In [ ]:
player_test

In [ ]:
DATA_copy['player_intro'].update(player_test)

In [ ]:
DATA_copy

## MongoDB Data Storage

In [31]:
#DECLARE WHAT NUMBER SCRAPE THIS IS
x = 6

In [1]:
import pymongo
from bson import ObjectId
import datetime

In [2]:
# Create connection variable
conn = 'mongodb://localhost:27017'

In [4]:
# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)

In [35]:
# Connect to a database. Will create one if not already available.
db = client.pga_data["collection"+f"{x}"]

In [36]:
now = datetime.datetime.now()

In [37]:
# create mongo collection
#collection = db.data

In [38]:
for d in DATA:
    dict_ = {"_id" : d['player_intro']['Player Name']}
    for x in d:
        dict_[f"{x}"]  = d[f"{x}"]
    try:
        db.insert_one(dict_)
    except:
        nothing = 0

In [ ]:
#x.inserted_id

In [ ]:
#bla = db.collection.find_one({'_id':ObjectId('5cd799a186fd7a260400d731')})
#bla

## Get Data from MongoDB

In [155]:
import pymongo
from bson import ObjectId
import datetime

In [134]:
conn2 = 'Cluster0-055w2.mongodb.net'

In [159]:
# Create connection variable
conn1 = 'mongodb://localhost:27017'

In [113]:
# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn1)

In [118]:
client2 = pymongo.MongoClient(conn2)

In [135]:
db = client.Dumpster_db['divers']

In [136]:
db = client2.list_database_names()

ServerSelectionTimeoutError: cluster0-055w2.mongodb.net:27017: [Errno 11001] getaddrinfo failed

In [ ]:
db

In [24]:
client2 = pymongo.MongoClient(conn2)

In [ ]:
# client.admin.command('copydb',
#                          fromdb='source_db_name',
#                          todb='target_db_name',
#                          fromhost='source.example.com')

In [160]:
# Connect to a database. Will create one if not already available.
db = client.pga_data["collection5"]

In [161]:
mongo_data = db.find()

In [9]:
pga_data_all = db.find()
player_intro = []
for player in pga_data_all:
    player_intro.append(player)

In [ ]:
default_data.update({'item3': 3})

In [51]:
data_cf = []
mongo_data = db.find()
for data in mongo_data:
    all_stat_var = data['all_stat_var']
    for stat in all_stat_var:
        stat.update({'player_name': data['_id']})
    data_cf.append(data['all_stat_var'])
data_cf

[[{'year': '2004',
   'Driving Distance': '281.5',
   'Driving Accuracy Percentage': '64.29%',
   'Right Rough Tendency': '21.43%',
   'Measured Rounds': '2',
   'Total Distance': '1,126',
   'Total Drives': '4',
   '# of Drives': '28',
   'Fairways Hit': '18',
   'Possible Fairways': '28',
   'Total Right rough': '6',
   'Possible Fwys': '28',
   'player_name': 'Adams, John'},
  {'year': '2003',
   'Total Distance': '1,097',
   'Total Drives': '4',
   'Fairways Hit': '17',
   'Possible Fairways': '28',
   '# of Drives': '28',
   'player_name': 'Adams, John'},
  {'year': '2002',
   'Total Distance': '1,193',
   'Total Drives': '4',
   'Fairways Hit': '13',
   'Possible Fairways': '28',
   'player_name': 'Adams, John'},
  {'year': '2000',
   'Total Distance': '2,175',
   'Total Drives': '8',
   'Fairways Hit': '34',
   'Possible Fairways': '56',
   'player_name': 'Adams, John'},
  {'year': '1998', 'Total Driving': '205', 'player_name': 'Adams, John'},
  {'year': '1997', 'Total Driving':

In [24]:
data_cf[0]

[{'year': '2004',
  'Driving Distance': '281.5',
  'Driving Accuracy Percentage': '64.29%',
  'Right Rough Tendency': '21.43%',
  'Measured Rounds': '2',
  'Total Distance': '1,126',
  'Total Drives': '4',
  '# of Drives': '28',
  'Fairways Hit': '18',
  'Possible Fairways': '28',
  'Total Right rough': '6',
  'Possible Fwys': '28',
  'player_name': 'Adams, John'},
 {'year': '2003',
  'Total Distance': '1,097',
  'Total Drives': '4',
  'Fairways Hit': '17',
  'Possible Fairways': '28',
  '# of Drives': '28',
  'player_name': 'Adams, John'},
 {'year': '2002',
  'Total Distance': '1,193',
  'Total Drives': '4',
  'Fairways Hit': '13',
  'Possible Fairways': '28',
  'player_name': 'Adams, John'},
 {'year': '2000',
  'Total Distance': '2,175',
  'Total Drives': '8',
  'Fairways Hit': '34',
  'Possible Fairways': '56',
  'player_name': 'Adams, John'},
 {'year': '1998', 'Total Driving': '205', 'player_name': 'Adams, John'},
 {'year': '1997', 'Total Driving': '190', 'player_name': 'Adams, Joh

Works

In [ ]:
test_data = db.find()

In [ ]:
for bla in range(0,10):
    print(test_data[bla]['player_intro'])

In [ ]:
test_data[1]['all_stat_var']

In [137]:
#test

In [4]:
from flask import Flask, render_template, redirect, jsonify
# Import our pymongo library, which lets us connect our Flask app to our Mongo database.
import pymongo
import os
import ssl
import json
from pymongo import MongoClient

#from flask import Flask
from flask_pymongo import PyMongo
from flask_sqlalchemy import SQLAlchemy

In [34]:
# create instance of Flask app
app = Flask(__name__)


# app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get('DATABASE_URL', '') or "sqlite:///db.sqlite"
# db = SQLAlchemy(app)

#ATTEMPT FOR HEROKU DEPLOYMENT#########################
MONGODB_URL = os.environ.get("MONGODB_URL")
client = pymongo.MongoClient(MONGODB_URL)
db = client.pga_data['collection5']
#######################################################
# app.config['MONGO_URI'] = os.environ.get("MONGODB_URL", '')
# db = SQLAlchemy(app)

#app.config["MONGO_URI"] = os.environ.get("MONGODB_URL", 'mongodb://localhost:27017/pga_data') or "pga_viz.json"
app.config["MONGO_URI"] = os.environ.get("MONGODB_URL", 'mongodb://localhost:27017/rest')
mongo = PyMongo(app)

In [39]:
import json
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['please_work']
collection_currency = db['currency']

In [44]:
with open('pga_viz.json') as f:
    file_data = json.load(f)

In [47]:
# use collection_currency.insert(file_data) if pymongo version < 3.0
collection_currency.insert_many(file_data)  
client.close()

In [35]:
mongo = mongo.db.find()

TypeError: 'Collection' object is not callable. If you meant to call the 'find' method on a 'Database' object it is failing because no such method exists.

In [32]:
for r in mongo:
    print(r)

TypeError: 'NoneType' object is not iterable

In [171]:
data_cf = []
for data in mongo_data:
    all_stat_var = data['all_stat_var']
    for stat in all_stat_var:
        stat.update({'player_name': data['_id']})
    data_cf.append(data['all_stat_var'])
data_cf

[]

In [1]:
import json

In [2]:
with open("pga_viz.json", "w") as write_file:
    json.dump(data, write_file)

NameError: name 'data' is not defined

In [3]:
json.loads()

NameError: name 'pga_viz' is not defined